In [26]:
cd '/content/drive/My Drive/machine_learning_cl/MovieReview/'

/content/drive/My Drive/machine_learning_cl/MovieReview


In [27]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
review_data = load_files(r"movie_review")
X, y = review_data.data, review_data.target

documents = []
stemmer = WordNetLemmatizer()

In [ ]:
for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [ ]:
import pickle

with open('documents.p', 'wb') as file:
    pickle.dump(documents, file)
    pickle.dump(y, file)

In [28]:
import pickle

with open('documents.p', 'rb') as file:    # weights.p 파일을 바이너리 읽기 모드(rb)로 열기
    documents = pickle.load(file)
    y = pickle.load(file)

사이킷

In [29]:
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

keras

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [ ]:
max_words = 500 # 실습에 사용할 단어의 최대 개수
num_classes = 1 # 레이블의 수

def prepare_data(data, mode): # 전처리 함수
    t = Tokenizer(num_words = max_words) # max_words 개수만큼의 단어만 사용한다.
    t.fit_on_texts(data)
    data = t.texts_to_matrix(data, mode=mode) # 샘플 수 × max_words 크기의 행렬 생성
    return data, t.index_word
    
data, index_to_word = prepare_data(documents, 'binary') 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, shuffle=False)

word2vec

In [ ]:
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

# tokenization
tokens = []
stop_words = set(stopwords.words('english')) 

for row in documents:
  word_tokens = word_tokenize(row)
  result =[word for word in word_tokens if not word in stop_words]
  tokens.append(result)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Word2Vec embedding
from gensim.models import Word2Vec
embedding_model = Word2Vec(tokens, size=1, window = 2, min_count=50, workers=4, iter=100, sg=1)

In [ ]:
# check embedding result
print(embedding_model.most_similar(positive=["movie"], topn=30))

[('constant', 1.0), ('featuring', 1.0), ('send', 1.0), ('mob', 1.0), ('100', 1.0), ('serve', 1.0), ('hint', 1.0), ('bored', 1.0), ('likable', 1.0), ('position', 1.0), ('routine', 1.0), ('hidden', 1.0), ('enemy', 1.0), ('ups', 1.0), ('goofy', 1.0), ('patch', 1.0), ('jedi', 1.0), ('current', 1.0), ('mulan', 1.0), ('ford', 1.0), ('faith', 1.0), ('professor', 1.0), ('hurt', 1.0), ('nomination', 1.0), ('wave', 1.0), ('humour', 1.0), ('succeeds', 1.0), ('pilot', 1.0), ('study', 1.0), ('rob', 1.0)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
embedding_model.save('Word2vec.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
 def Convert2Vec(model_name, doc):  ## Convert corpus into vectors
        word_vec = []
        model = Word2Vec.load(model_name)
        for sent in doc:
            sub = []
            length = len(sent)
            for i in range(58): 
              if(i < length):
                word = sent[i]
                if (word in model.wv.vocab):
                  sub.append(model.wv[word])
                else :
                  sub.append(np.random.rand(1)) ## used for OOV words
              else:
                  sub.append(0.0) 
            word_vec.append(sub)
        return np.array(word_vec)

In [ ]:
entire = Convert2Vec("./Word2vec.model",tokens)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


##Split data to (train images, train labels) and (test images, test labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(entire, y, test_size=0.3, shuffle=False)

In [ ]:
np.savez('train_test', X_train = X_train, X_test = X_test, y_train=y_train, y_test=y_test)

In [ ]:
import numpy as np
train_testload = np.load('./train_test.npz', allow_pickle=True)
X_train= train_testload['X_train']
X_test= train_testload['X_test']
y_train= train_testload['y_train']
y_test= train_testload['y_test']

In [ ]:
X_train = np.array(X_train, np.float)
X_test = np.array(X_test, np.float)
y_train = np.array(y_train, np.float)
y_test = np.array(y_test, np.float)

In [ ]:
# create label for one hot encoding
def label(num):
  label = np.zeros(10, dtype=int)
  label[num] = 1
  return label

def labels():
  labels = np.zeros((10,10), dtype = int)
  for i in range(10):
    labels[i, :] = label(i)
  return labels

l = labels() #10X10

In [30]:
## NN architecture
numX = 1500
numY = 255
numZ = 18
numH = 1

In [31]:
import numpy as np
#가장 처음 한 번만 실행
# initializing model parameters

np.random.seed(seed=100) #랜덤값 고정

U1d = np.random.normal(size=numY*(numX+1))
V1d = np.random.normal(size=numZ*(numY+1))
W1d = np.random.normal(size=numH*(numZ+1))
past_itr = 0


In [32]:
accuracy_array = np.zeros(past_itr, dtype =float)
cost_array = np.zeros(past_itr, dtype =float)

In [33]:
#이미지 1개
from numba import jit
@jit(nopython=True, parallel=True)
def J(H, label): # 10*1 10*1
  cost = (-label)*(np.log(H)) 
  - (1-label)*(np.log(1-H))
  SUM = np.sum(cost)
  return SUM

from numba import jit
@jit(nopython=True)
def zeros(length):
    return np.zeros(length, np.float64)  # np.float64 instead of np.float

@jit(nopython=True)
def zerosShape(row, col):
    return np.zeros((row, col), np.float64)  # np.float64 instead of np.float

@jit(nopython=True)
def zeros(length):
    return np.zeros(length, np.float64)  # np.float64 instead of np.float

@jit(nopython=True)
def empty(size):
    return np.empty(size, np.float64)

@jit(nopython=True, parallel=True)
def sigma(X):
    return 1.0/(1.0+(np.exp(-X)))

@jit(nopython=True, parallel=True)
def square(X):
    return np.square(X)

@jit(nopython=True, parallel=True)
def sum_square(X):
  acc = 0
  for val in X:
    val2 = np.square(val)
    acc += np.sum(val2)
  return acc

@jit
def Savez(save_info1, save_info2, save_info3):
  np.savez("file_info", info1=save_info1, info2=save_info2, info3=save_info3) 

In [34]:
from numba import njit
itr = 50

##model
@jit(nopython=False)
def model(U1d, V1d, W1d, past_itr, itr, numvectors, accuracy_array_t, cost_array_t, X_train, y_train ):
  #gradient descent
  predicted_label = zeros(numvectors)
  lamb = 5

  ##코드를 여러번 돌릴 때마다 accuracy array와 cost array 길이 늘리기
  accuracy_array = zeros(past_itr+itr)
  accuracy_array[:past_itr] = accuracy_array_t[:past_itr]
  cost_array = zeros(past_itr+itr)
  cost_array[:past_itr] = cost_array_t[:past_itr]
  ln_rate = 0.1

  save_info1 = zerosShape(numY*(numX+1), itr)
  save_info2 = zerosShape(numZ*(numY+1), itr)
  save_info3 = zerosShape(numH*(numZ+1), itr)
  
  for j in range(itr):#iteration 
    
    j = past_itr + j  #cost와 accuracy array의 index
    errors = 0 
    ifzero = 1 

    save_info1[:, j] = U1d #save from the a array
    save_info2[:, j] = V1d #save from the b array
    save_info3[:, j] = W1d

    if (j==0):
      ifzero = 0
    #if (j%30==0):
     # ln_rate = ln_rate * (0.002)
    print("iteration:", j+1)

    for i in range(numvectors):

      ## resizing 
      U = U1d.reshape(numX+1, numY).copy()
      V = V1d.reshape(numY+1, numZ).copy()
      W = W1d.reshape(numZ+1, numH).copy()

      #print(i)
      X  = empty((numX, 1))
      xf = X_train[i].flatten()
      vec = np.ascontiguousarray(xf)
      X = vec.reshape(((numX), 1) )
      label = y_train[i] #int 
      Xt_nob = np.transpose(X)

      #  print("Xt_nob : ", Xt_nob)
      X0 = np.array([1]) #bias
      X_b = np.append(vec, X0)
      Xt = X_b.reshape(1,numX+1) 
      #  print("Xt : ", Xt)
      Y = np.dot(Xt, U)

      #activation function
      ##sigma = lambda x: 1/(1+(np.exp(-x)))
      ##vsigma = np.vectorize(sigma)

      Y_tilda_nob = sigma(Y)
      #  print("Y_tilda_nob : ", Y_tilda_nob)
      Y0 = np.array([1])
      vec_y = np.ascontiguousarray(Y_tilda_nob[0, :])
      Y_tilda_b = np.append(vec_y, Y0)
      Y_tilda = Y_tilda_b.reshape(1,numY+1)
      #  print("Y_tilda : ", Y_tilda)

      Z = np.dot(Y_tilda, V)
      Z_tilda_nob = sigma(Z)
      #  print("Z_tilda_nob : ", Z_tilda_nob)

      Z0 = np.array([1])
      vec_z = np.ascontiguousarray(Z_tilda_nob[0, :])
      Z_tilda_b = np.append(vec_z, Z0)
      Z_tilda = Z_tilda_b.reshape(1,numZ+1)
      #  print("Z_tilda : ", Z_tilda)
      H = np.dot(Z_tilda, W)
      H_tilda = sigma(H)
      #  print("H_tilda : ", H_tilda)

      ##error
      error = J(H_tilda, label)
      errors += error
      if(H_tilda.flatten()[0] >= 0.5):
        predicted_label[i] = 1
      else:
        predicted_label[i] = 0
      
      #print(" error : ", error)
      #print(" predicted_label[i]", predicted_label[i])
      #print(" train_label", train_label[i])

      ##back propagation
      ## NN architecture

      ## W - 업데이트 49 X 10 - no bias 50*10 #Ztilda는 49개의 w 업데이트의 과정에서 고정
      Z_tilda_nob = Z_tilda_nob.reshape(numZ,1)
      redundant = (H_tilda - label).reshape(1,numH ) #1x10
      if(j== 0): 
        W[:numZ, :] = W[:numZ, :]- ln_rate * np.dot(Z_tilda_nob, redundant)
      W[:numZ, :] = W[:numZ, :]*(1-ln_rate*lamb/numvectors)- ln_rate * np.dot(Z_tilda_nob, redundant) # 49x10
      #print(W.shape)

      ## V - 업데이트 196X49
      z = sigma(Z)*(1-sigma(Z))
      Y_tilda_nob = Y_tilda_nob.reshape(numY,1)
      w = np.ascontiguousarray(W[:numZ, :])
      a = np.dot(redundant, w.reshape(numH,numZ))
      b = z
      redundant2 = (a*b) 
      if(j== 0):
        V[:numY, :] = (V[:numY, :] - ln_rate * np.dot(Y_tilda_nob, redundant2))              
      V[:numY, :] = (V[:numY, :]*(1-ln_rate*lamb/numvectors) - ln_rate * np.dot(Y_tilda_nob, redundant2))                     #196X49
      #print(V.shape)
      
      ## U - 업데이트
      y = sigma(Y)*(1-sigma(Y))
      Xt_nob = Xt_nob.reshape(numX,1)
      v =  np.ascontiguousarray(V[:numY, :])
      c = np.dot(redundant2, v.reshape(numZ,numY))
      d = y
      redundant3 = (c*d)          #1x196
      if(j== 0):
        U[:numX, :] = (U[:numX, :] - ln_rate * np.dot(Xt_nob,redundant3))
      U[:numX, :] = (U[:numX, :]*(1-ln_rate*lamb/numvectors) - ln_rate * np.dot(Xt_nob,redundant3))
      #print("U : ", U.shape)

      U1d = U.flatten()
      V1d = V.flatten()
      W1d = W.flatten()

    cost = errors/numvectors + ifzero * (lamb/(2*numvectors))*sum_square([vec,vec_y,vec_z])
    ## accuracy
    ## check if the prediction is correct
    count = 0
    for real, hypo in zip(y_train, predicted_label):
        if real == hypo:
          count = count + 1
      
    accuracy = count/numvectors
      
    print("cost ", cost)
    print("accuracy ", accuracy)
    accuracy_array[j] = accuracy 
    
    cost_array[j] = cost
  past_itr = j + 1
  
  np.savez("file_info", info1=save_info1, info2=save_info2, info3=save_info3)
  return past_itr,accuracy_array,cost_array,predicted_label
 

In [ ]:
training_itr,accuracy_array,cost_array,predicted_label_train=model(U1d, V1d, W1d, past_itr, itr, 1401, accuracy_array, cost_array, X_train, y_train )

<ipython-input-34-7a16c4621ae9>:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "model" failed type inference due to: Use of unsupported NumPy function 'numpy.savez' or unsupported use of the function.

File "<ipython-input-34-7a16c4621ae9>", line 153:
def model(U1d, V1d, W1d, past_itr, itr, numvectors, accuracy_array_t, cost_array_t, X_train, y_train ):
    <source elided>
  
  np.savez("file_info", info1=save_info1, info2=save_info2, info3=save_info3)
  ^

[1] During: typing of get attribute at <ipython-input-34-7a16c4621ae9> (153)

File "<ipython-input-34-7a16c4621ae9>", line 153:
def model(U1d, V1d, W1d, past_itr, itr, numvectors, accuracy_array_t, cost_array_t, X_train, y_train ):
    <source elided>
  
  np.savez("file_info", info1=save_info1, info2=save_info2, info3=save_info3)
  ^

  @jit(nopython=False)
<ipython-input-34-7a16c4621ae9>:5: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled 

iteration: 1
cost  0.4729140062083214
accuracy  0.4989293361884368
iteration: 2
cost  0.556903908626406
accuracy  0.4953604568165596
iteration: 3
cost  0.560085778276884
accuracy  0.4939329050678087
iteration: 4
cost  0.5690580583296907
accuracy  0.49321912919343325
iteration: 5
cost  0.5771256515565821
accuracy  0.49464668094218417
iteration: 6
cost  0.5885155544012031
accuracy  0.48822269807280516
iteration: 7
cost  0.589346949709569
accuracy  0.489650249821556
iteration: 8
cost  0.5893933472341496
accuracy  0.489650249821556
iteration: 9
cost  0.5894175754334982
accuracy  0.489650249821556
iteration: 10
cost  0.5894327204295957
accuracy  0.489650249821556
iteration: 11
cost  0.5894421967996247
accuracy  0.489650249821556
iteration: 12


In [ ]:
print(predicted_label_train)

1401


In [ ]:
classifier = train_classifier(X_train, y_train)
y_pred_test = classifier.predict(X_test)
y_pred_train = classifier.predict(X_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))
print(accuracy_score(y_train, y_pred_train))

print(confusion_matrix(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))
print(accuracy_score(y_test, y_pred_test)
